In [11]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)



Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [20]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002

n_hidden = 256
n_hidden2 = 512
n_input = 28 * 28
n_noise = 128

In [21]:
X = tf.placeholder(tf.float32, [None, n_input])
z = tf.placeholder(tf.float32, [None, n_noise])

# 생성기
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))

G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

'''
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_hidden2], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_hidden2]))

G_W3 = tf.Variable(tf.random_normal([n_hidden2, n_input], stddev=0.01))
G_b3 = tf.Variable(tf.zeros([n_input]))
'''

# 판별기

D_W1 = tf.Variable(tf.random_normal([n_input,n_hidden], stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))
'''
D_W2 = tf.Variable(tf.random_normal([n_hidden,n_noise], stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([n_noise]))

D_W3 = tf.Variable(tf.random_normal([n_noise, 1], stddev=0.01))
D_b3 = tf.Variable(tf.zeros([1]))
'''

'\nD_W2 = tf.Variable(tf.random_normal([n_hidden,n_noise], stddev = 0.01))\nD_b2 = tf.Variable(tf.zeros([n_noise]))\n\nD_W3 = tf.Variable(tf.random_normal([n_noise, 1], stddev=0.01))\nD_b3 = tf.Variable(tf.zeros([1]))\n'

In [22]:
# 생성기 신경망

def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    #hidden2 = tf.nn.relu(tf.matmul(hidden, G_W2) + G_b2)
    #output = tf.nn.sigmoid(tf.matmul(hidden2,G_W3) + G_b3)
    
    return output

# 판별기 신경망

def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    #hidden2 = tf.nn.relu(tf.matmul(hidden, D_W2) + D_b2)
    #output = tf.nn.sigmoid(tf.matmul(hidden2, D_W3) + D_b3)
    
    return output

# 랜덤한 노이즈만들기
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [23]:
G = generator(z)
D_gene = discriminator(G)

D_real = discriminator(X)

In [24]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(1-D_gene))

In [25]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [26]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                         var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G,
                                                         var_list=G_var_list)

In [27]:
### 모델 학습

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D , loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        # 판별기와 생성기 신경망을 각각 학습시킨다.
        _, loss_val_D = sess.run([train_D, loss_D], 
                                 feed_dict = {X: batch_xs, z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], 
                                 feed_dict = {z: noise})
        
    print('Epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    
    ### 학습이 이뤄지는거 확인하기위해서 이미지 생성
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict = {z: noise})
        
        fig, ax = plt.subplots(1, sample_size , figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('samples_test/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        
print('끝!')

Epoch: 0000 D loss: -0.4334 G loss: -0.1247
Epoch: 0001 D loss: -0.1769 G loss: -0.05948
Epoch: 0002 D loss: -0.07795 G loss: -0.0337
Epoch: 0003 D loss: -0.3234 G loss: -0.1262
Epoch: 0004 D loss: -0.2211 G loss: -0.09372
Epoch: 0005 D loss: -0.2062 G loss: -0.1184
Epoch: 0006 D loss: -0.1451 G loss: -0.08401
Epoch: 0007 D loss: -0.2989 G loss: -0.1254
Epoch: 0008 D loss: -0.2087 G loss: -0.07888
Epoch: 0009 D loss: -0.1123 G loss: -0.042
Epoch: 0010 D loss: -0.6338 G loss: -0.1998
Epoch: 0011 D loss: -0.3141 G loss: -0.08525
Epoch: 0012 D loss: -0.2316 G loss: -0.07126
Epoch: 0013 D loss: -0.3623 G loss: -0.1676
Epoch: 0014 D loss: -0.08978 G loss: -0.04069
Epoch: 0015 D loss: -0.3496 G loss: -0.1397
Epoch: 0016 D loss: -0.2509 G loss: -0.09402
Epoch: 0017 D loss: -0.1767 G loss: -0.05096
Epoch: 0018 D loss: -0.1677 G loss: -0.05724
Epoch: 0019 D loss: -0.1513 G loss: -0.06243
Epoch: 0020 D loss: -0.2277 G loss: -0.07291
Epoch: 0021 D loss: -0.07804 G loss: -0.01962
Epoch: 0022 D los

In [13]:
1. 신경망 추가하기
2. 데이터 바꿔보기 등등

SyntaxError: invalid syntax (<ipython-input-13-6700090ffbe5>, line 1)